In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 划分训练集和数据预处理

## 数据预处理

In [ ]:
data=pd.read_csv('C:/Users/nigel/Desktop/kaggle competition/titanic/data.csv')#导入数据
cols = data.columns[data.columns != 'Survived']
y=pd.DataFrame(data['Survived'])#y为目标变量
x=data[cols]
#转化格式
data['Survived']=data['Survived'].astype('category',errors='ignore')#转换类型

## 划分训练集

In [3]:
import random
import os
from sklearn.model_selection import train_test_split
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    os.environ["PYTHONHASHSEED"] = str(seed_value)
    

SEED = 42
set_seed(SEED)
#划分训练集
#x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=,trian_size=,random_state,shuffle,stratify)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=SEED)

### 标准化和归一化

In [ ]:
#标准化
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
normaler = Normalizer()
scaler = StandardScaler()
cols =x_train.columns
x_train[cols] = scaler.fit_transform(x_train[cols])
x_train[cols] = normaler.fit_transform(x_train[cols])
x_test[cols] = scaler.transform(x_test[cols])
x_test[cols] = normaler.transform(x_test[cols])

# 训练模型


## 创建学习器 交叉验证选择超参数

不同模型只需要更换学习器和参数

### Decision Tree

In [ ]:
import time
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn import tree
cross_valid_scores = {}
start=time.time()
parameters_tree = {
    "criterion": ["gini", "entropy"],#"log_loss"
    "splitter": ["best","random"],
    'random_state':[SEED],
    'max_depth':[i for i in np.arange(3,20,step=1,dtype=int)]
    #"max_leaf_nodes":float,default=0
    #"max_depth": int,
    #"min_samples_split":int ,default=2
    #"min_samples_leaf":int,default=1
    #"min_weight_fraction_leaf:"float,default=0
    #"max_featuresint": float or {“auto”, “sqrt”, “log2”}, default=None
    #"class_weight":dict, list of dict or “balanced”, default=None
    #"ccp_alpha":non-negative float, default=0.0
}
model_tree=tree.DecisionTreeClassifier()

model_gridSearch_tree=GridSearchCV(model_tree,parameters_tree,cv=10,scoring='accuracy',verbose=1,return_train_score=True,n_jobs=-1)
#model_svc.fit(x_train, y_train.values.ravel(),eval_set=[(x_test,y_test),(x_train,y_train)])
model_gridSearch_tree.fit(x_train, y_train.values.ravel())
end=time.time()
print('-----')
print("Time=",end-start)
print(f'Best parameters {model_gridSearch_tree.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_gridSearch_tree.best_score_:.3f}'
)
cross_valid_scores['tree'] = model_gridSearch_tree.best_score_
print('-----')

In [ ]:
model_tree.set_params(**model_gridSearch_tree.best_params_)
model_tree.get_params()


In [ ]:
model_tree.fit(x_train, y_train.values.ravel())

### logistics

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html  
如果想获取模型拟合后的p值等，需要用statsmodels包

solver超参数  
Algorithm to use in the optimization problem. Default is ‘lbfgs’. To choose a solver, you might want to consider the following aspects:  
1.For small datasets, ‘liblinear’ is a good choice, whereas ‘sag’ and ‘saga’ are faster for large ones;  
2.For multiclass problems, only ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ handle multinomial loss;  
3.‘liblinear’ is limited to one-versus-rest schemes.  
  
  
正则化方法和solver超参数的兼容性  
Warning The choice of the algorithm depends on the penalty chosen: Supported penalties by solver:  
1.‘newton-cg’ - [‘l2’, ‘none’]  
2.‘lbfgs’ - [‘l2’, ‘none’]  
3.‘liblinear’ - [‘l1’, ‘l2’]  
4.‘sag’ - [‘l2’, ‘none’]  
5.‘saga’ - [‘elasticnet’, ‘l1’, ‘l2’, ‘none’]  

In [ ]:
import time
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
cross_valid_scores = {}
start=time.time()
parameters_lg = {
    "penalty": ['l1', 'l2', 'elasticne', 'none'],#惩罚项，elasticne表示l1,l2
    "C": [i for i in np.arange(0,10,0.5)],#正则化参数的倒数，越小正则化越强
    'solver':['liblinear'],
    'max_iter':[200],
    'random_state':[SEED],
    #'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    #multi_class{‘auto’, ‘ovr’, ‘multinomial’}, default=’auto’，非常有趣的一个超参数
    #max_iter:int,default=100
}
model_lg=LogisticRegression()

model_gridSearch_lg=GridSearchCV(model_lg,parameters_lg,cv=10,scoring='accuracy',verbose=1,return_train_score=True,n_jobs=-1)
#model_svc.fit(x_train, y_train.values.ravel(),eval_set=[(x_test,y_test),(x_train,y_train)])
model_gridSearch_lg.fit(x_train, y_train.values.ravel())
end=time.time()
print('-----')
print("Time=",end-start)
print(f'Best parameters {model_gridSearch_lg.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_gridSearch_lg.best_score_:.3f}'
)
cross_valid_scores['logistics'] = model_gridSearch_lg.best_score_
print('-----')

In [ ]:
model_lg.set_params(**model_gridSearch_lg.best_params_)
model_lg.get_params()

In [ ]:
model_lg.fit(x_train, y_train.values.ravel())

### SVM

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [ ]:
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
cross_valid_scores = {}
start=time.time()
parameters_svc = {
    "C": [i for i in np.arange(0.1,0.2,step=0.01)],
    "kernel": ["linear","sigmoid"],# "poly", "rbf", sigmoid
    "gamma": ["scale", "auto"],
    'random_state':[SEED]
    #degreeint, default=3，当为poly时，多项式的度数，其他时则忽略
    #max_iterint, default=-1,迭代次数，-1表示无限制
}

model_svc = SVC(
    random_state=SEED,
    class_weight="balanced",
    probability=True,
)

model_gridSearch_svm = GridSearchCV(
    model_svc, 
    parameters_svc, 
    cv=10,
    scoring='accuracy',
    verbose=3,
    n_jobs=-1
)

model_gridSearch_svm=model_gridSearch_svm.fit(x_train, y_train.values.ravel())
end=time.time()
print('-----')
print("Time=",end-start)
print(f'Best parameters {model_gridSearch_svm.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_gridSearch_svm.best_score_:.3f}'
)
cross_valid_scores['svm'] = model_gridSearch_svm.best_score_
print('-----')

In [ ]:
model_svc.set_params(**model_gridSearch_svm.best_params_)
model_svc.get_params()

In [ ]:
model_svc.fit(x_train, y_train.values.ravel())

### KNN

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [ ]:
import time
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.model_selection import train_test_split, GridSearchCV
cross_valid_scores = {}
start=time.time()
parameters_knn = {
    "n_neighbors": [i for i in np.arange(1,10,dtype=int)],
    "leaf_size":[i for i in np.arange(10,50,dtype=int)],
    'p':[1,2]# 1 表示曼哈顿距离，2表示欧氏距离，其他的值，minkowski_distance
    #weights{‘uniform’, ‘distance’} or callable, default=’uniform’
    #algorithm{‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’
    #metric：str or callable, default=’minkowski’，控制距离度量，也可以自定义计算方式，或者传入已经计算好的距离
    #metric_params：dict, default=None，将参数传入metric超参数中的自定义函数
}

model_knn = knn(
)

model_gridSearch_knn = GridSearchCV(
    model_knn, 
    parameters_knn, 
    cv=10,
    scoring='accuracy',
    verbose=3,
    n_jobs=-1
)

model_gridSearch_knn=model_gridSearch_knn.fit(x_train, y_train.values.ravel())
end=time.time()
print('-----')
print("Time=",end-start)
print(f'Best parameters {model_gridSearch_knn.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_gridSearch_knn.best_score_:.3f}'
)
cross_valid_scores['knn'] = model_gridSearch_knn.best_score_
print('-----')

In [ ]:
model_knn.set_params(**model_gridSearch_knn.best_params_)
model_knn.get_params()

In [ ]:
model_knn.fit(x_train, y_train.values.ravel())

### XGBoost

Sklearn API:https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
cross_valid_scores = {}
import time
start=time.time()
parameters = {
    'max_depth': [3, 5, 7, 9], 
    'n_estimators': [5, 10, 15, 20, 25, 50, 100],
    'learning_rate': [0.01, 0.05, 0.1],
    'gamma':[0],
    'max_delta_step':[0],
    'min_child_weight':[1],
    'min_split_loss':[0.01],
    'eval_metric':['error']
    
}

model_xgb = xgb.XGBClassifier(
    random_state=SEED,
    objective='binary:logistic',
    tree_method='gpu_hist',
    predictor='gpu_predictor',n_jobs=-1,
    enable_categorical=True,
    max_cat_to_onehot=2

)

model_gridSearch_xgb = GridSearchCV(
    model_xgb, 
    parameters, 
    cv=10,
    scoring='accuracy',
    verbose=3
)

model_gridSearch_xgb =model_gridSearch_xgb.fit(x_train, y_train.values.ravel())
end=time.time()
print('-----')
print("Time=",end-start)
print(f'Best parameters {model_gridSearch_xgb .best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_gridSearch_xgb .best_score_:.3f}'
)
cross_valid_scores['xgboost'] = model_gridSearch_xgb .best_score_
print('-----')
### logistics

In [ ]:
model_xgb.set_params(**model_gridSearch_xgb.best_params_)
model_xgb.get_params()

In [ ]:
model_xgb.fit(x_train,y_train,eval_set=[(x_test, y_test), (x_train, y_train)])

In [ ]:
#预测为概率
#y_score = model_xgb.predict_proba(x_test)[:, 1]

In [ ]:
# Get a graph
#graph = xgb.to_graphviz(model_xgb, num_trees=1)
# Or get a matplotlib axis
#ax = xgb.plot_tree(model_xgb, num_trees=1)
# Get feature importances
model_xgb.feature_importances_
model_xgb.feature_names_in_

In [ ]:
#保存模型
model_xgb.save_model("categorical-model.json")
#读取模型
model_xgb=model_xgb.load_model("categorical-model.json")

## 通过kfold获得各个模型在相同数据上的效果，比较选择模型

In [ ]:
#保存将要比较的模型
#传递的是设置好超参数的模型
models={
    'svm':model_svc,
    'Xgboost':model_xgb
}

from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import roc_auc_score
## Import the library and functions you need
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
## Accuracy
#print("Accuracy:",accuracy_score(y_test,pred))
## Precision
#print("Precision:",precision_score(y_test,pred))
## Recall
#print("Recall:",recall_score(y_test,pred))
## F1 Score
#print("F1:",f1_score(y_test,pred))
#设置函数
def kf_cross_val(model,X,y):
    
    feature_imp, features = [],[]

    scores = pd.DataFrame(columns=['Accuracy','Precision','Recall','f1','roc_auc'])
    
    kf = KFold(n_splits=5,shuffle = True, random_state=42)
    
    for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
        
        x_train = X.loc[train_index]
        y_train = y.loc[train_index]
        x_test = X.loc[test_index]
        y_test = y.loc[test_index]
        
        model.fit(x_train,y_train.values.ravel())
        
        #y_pred = model.predict_proba(x_test)[:,1]     # edit 
        y_pred = model.predict(x_test)   # edit 

        roc_auc=roc_auc_score(y_test,y_pred)#roc曲线auc值
        f1=f1_score(y_test,y_pred)
        Recall=recall_score(y_test,y_pred)
        Precision=precision_score(y_test,y_pred)
        Accuracy=accuracy_score(y_test,y_pred)
        metrics=np.array([Accuracy,Precision,Recall,f1,roc_auc]).reshape(1,5)
        addto=pd.DataFrame(metrics,columns=['Accuracy','Precision','Recall','f1','roc_auc'],index=[fold])
        scores=pd.concat([scores,addto])


        
        
        try:
            feature_imp.append(model.feature_importances_)
            features.append(model.feature_names_)
        except AttributeError: # if model does not have .feature_importances_ attribute
            pass
        
    return feature_imp, scores, features
 
results  = {}
 
for name,model in models.items():
    
    feature_imp,result,features = kf_cross_val(model, x, y)
    results[name] = result
 
for name, result in results.items():
    print("----------\n" + name)
    print(np.mean(result))
    print(np.std(result))
    print(feature_imp)



# Stacking

将之前设置好的学习器传入
有两种stack： 1、每个学习器都设置好了超参数，则直接进行训练；2、在stacking中一起进行超参数选择


In [17]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
from mlxtend.regressor import StackingCVRegressor
from mlxtend.classifier import StackingCVClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
stack = StackingCVClassifier(classifiers=[model_xgb,model_svc],
                            meta_classifier=lr, cv=10,
                            use_features_in_secondary=False,
                            store_train_meta_features=True,
                            shuffle=False,n_jobs=2)

In [20]:
for clf, label in zip([model_xgb,model_svc,stack], 
                      ['xgb', 
                       'svc', 
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf,x_train , y_train.values.ravel(),cv=10, scoring='accuracy',n_jobs=-1)
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

Accuracy: 1.00 (+/- 0.00) [xgb]
Accuracy: 1.00 (+/- 0.00) [svc]
Accuracy: 1.00 (+/- 0.00) [StackingClassifier]


In [21]:
result=stack.fit(x_train.values ,y_train.values.ravel())

d:\python\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
d:\python\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


# 检测效果

In [ ]:
from sklearn import metrics
pred = stack.predict(x_test)
confusion_matrix = metrics.confusion_matrix(y_test, pred)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
cm_display.plot()
plt.show()


In [ ]:
## Import the library and functions you need
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
## Accuracy
print("Accuracy:",accuracy_score(y_test,pred))
## Precision
print("Precision:",precision_score(y_test,pred))
## Recall
print("Recall:",recall_score(y_test,pred))
## F1 Score
print("F1:",f1_score(y_test,pred))

# scikit-optimize

## BayesSearchCV

自定义程度更低
https://scikit-optimize.github.io/stable/auto_examples/sklearn-gridsearchcv-replacement.html

### SVM

#### 设置搜索空间

In [7]:
# define search space
#svm
params = dict()
params['C'] = (1e-6, 100.0, 'log-uniform')
params['gamma'] = (1e-6, 100.0, 'log-uniform')
params['degree'] = (1,5)
params['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

In [41]:
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV
from sklearn.svm import SVC
# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=SEED)
# define the search
search = BayesSearchCV(estimator=SVC(),scoring=evaluate_model,search_spaces=params, n_jobs=-1, cv=cv)
#search = BayesSearchCV(estimator=SVC(),search_spaces=params, n_jobs=-1, cv=cv)

In [40]:
def evaluate_model(estimator,X,y):
    # configure the model with specific hyperparameters
    acc = cross_val_score(estimator, X, y, scoring="accuracy",cv=cv).mean()#评测分数是cv后的结果
    return acc

In [ ]:
# perform the search
search.fit(x_train, y_train)
# report the best result
print(search.best_score_)
print(search.best_params_)
print(search.cv_results_)

## Bayes optimization

自定义程度更高，可以自定义目标函数、surrogate函数、acquistion函数

### SVM

#### 定义搜索空间

In [28]:
# define the space of hyperparameters to search
# XGboost
SPACE = [
   skopt.space.Real(0.01, 0.5, name='learning_rate', prior='log-uniform'),
   skopt.space.Integer(1, 30, name='max_depth'),
   skopt.space.Integer(2, 100, name='num_leaves'),
   skopt.space.Integer(10, 1000, name='min_data_in_leaf'),
   skopt.space.Real(0.1, 1.0, name='feature_fraction', prior='uniform'),
   skopt.space.Real(0.1, 1.0, name='subsample', prior='uniform')]

In [45]:
import skopt
#SVM
search_space = [
   skopt.space.Real(0.01, 0.5, name='C', prior='log-uniform'),
   skopt.space.Integer(1, 5, name='degree'),
   skopt.space.Real(0.001,10 ,name='gamma'),
   skopt.space.space.Categorical(['linear', 'poly', 'rbf', 'sigmoid'],name='kernel')]
   

#### 目标函数

In [46]:
# 为什么是负号？
#因为scikit-optimize的训练过程是，使得目标函数最小。如果设置的是acc，则应该反过来
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
@skopt.utils.use_named_args(search_space)
def evaluate_model(**params):
    # configure the model with specific hyperparameters
    clf = SVC(**params)
    acc = cross_val_score(clf, x_train, y_train, scoring="accuracy",cv=5).mean()#评测分数是cv后的结果
    return -acc


#### 选择surrogate函数，超参数寻优

In [ ]:
#可选有：gp_minimize,dummy_minimize,forest_minimize,gbrt_minimize
#参数：https://scikit-optimize.github.io/stable/modules/generated/skopt.gp_minimize.html

from skopt import gp_minimize
result = gp_minimize(
    func=evaluate_model,
    dimensions=search_space,
    acq_func="gp_hedge",#acquisition函数
    n_calls=100,
    random_state=SEED,
    verbose=True,
    n_jobs=-1,
)

#### result

In [ ]:
# summarizing finding:

print('Best Accuracy: %.3f' % (result.fun)) 
print('Best Parameters: %s' % (result.x))`b

In [ ]:
from skopt.plots import plot_convergence,plot_evaluations,plot_objective,plot_regret

plot_convergence(result)
plot_evaluations(result)
plot_objective(result)
plot_regret(result)

### XGBoost

好像只支持sklearn里面的函数，所以得有sklearn 的 API

In [41]:
# define the space of hyperparameters to search
# XGboost
search_space = [
   skopt.space.Real(0.01, 0.5, name='eta', prior='log-uniform'),
   skopt.space.Integer(1, 30, name='max_depth'),
   skopt.space.Integer(2, 100, name='num_leaves'),
   skopt.space.Integer(10, 1000, name='min_data_in_leaf'),
   skopt.space.Real(0.1, 1.0, name='subsample', prior='uniform')]

In [54]:
# 为什么是负号？
#因为scikit-optimize的训练过程是，使得目标函数最小。如果设置的是acc，则应该反过来
from sklearn.model_selection import cross_val_score
import xgboost as xgb

@skopt.utils.use_named_args(search_space)
def evaluate_model(**params):
    # configure the model with specific hyperparameters
    clf = xgb.XGBClassifier(**params)
    acc = cross_val_score(clf, x_train, y_train, scoring="accuracy",cv=5).mean()#评测分数是cv后的结果
    return -acc



使用Neptune
放弃了，因为没有适合新版本的

In [ ]:
from skopt import gp_minimize

result = gp_minimize(
    func=evaluate_model,
    dimensions=search_space,
    acq_func="gp_hedge",#acquisition函数
    n_calls=100,
    random_state=SEED,
    verbose=True,
    n_jobs=-1,
)

In [ ]:
# summarizing finding:

print('Best Accuracy: %.3f' % (result.fun)) 
print('Best Parameters: %s' % (result.x))